In [48]:
import numpy as np
import pandas as pd
# from tqdm import tqdm, tqdm_notebook
from tqdm import trange, tqdm_notebook

import seaborn as sns
from matplotlib import pyplot as plt
from geopy import distance

Read train data + preprocess

In [34]:
# read train data + preprocess + save df_cleaned_train
df = pd.read_csv('train.csv', sep=',')
num_points, num_attributes = df.shape
print("There are {} data points, each with {} attributes.". format(num_points, num_attributes))

There are 431732 data points, each with 17 attributes.


In [35]:
# check if there is missing value for each attribute
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431732 entries, 0 to 431731
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                431732 non-null  object 
 1   town                 431732 non-null  object 
 2   flat_type            431732 non-null  object 
 3   block                431732 non-null  object 
 4   street_name          431732 non-null  object 
 5   storey_range         431732 non-null  object 
 6   floor_area_sqm       431732 non-null  float64
 7   flat_model           431732 non-null  object 
 8   eco_category         431732 non-null  object 
 9   lease_commence_date  431732 non-null  int64  
 10  latitude             431732 non-null  float64
 11  longitude            431732 non-null  float64
 12  elevation            431732 non-null  float64
 13  subzone              431732 non-null  object 
 14  planning_area        431732 non-null  object 
 15  region           

In [36]:
# remove eco_category, elevation, block, region, and planning_area
irrel_redun_columns = ['eco_category', 'elevation', 'block', 'region', 'planning_area']
df = df.drop(irrel_redun_columns, axis=1)

In [37]:
# clean the ambiguity of column 'flat_type' and 'street_name'
df['flat_type'] = df['flat_type'].replace({'1 room': '1-room', '2 room': '2-room', '3 room': '3-room', '4 room': '4-room', '5 room': '5-room'})
df['street_name'] = df['street_name'].str.title()

In [38]:
# save the train_cleaned.csv
df.to_csv('train_cleaned.csv', index = False)

Read test data + preprocess

In [39]:
# read train data + preprocess + save df_cleaned_train
df = pd.read_csv('test.csv', sep=',')
num_points, num_attributes = df.shape
print("There are {} data points, each with {} attributes.". format(num_points, num_attributes))

There are 107934 data points, each with 16 attributes.


In [40]:
# check if there is missing value for each attribute
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107934 entries, 0 to 107933
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                107934 non-null  object 
 1   town                 107934 non-null  object 
 2   flat_type            107934 non-null  object 
 3   block                107934 non-null  object 
 4   street_name          107934 non-null  object 
 5   storey_range         107934 non-null  object 
 6   floor_area_sqm       107934 non-null  float64
 7   flat_model           107934 non-null  object 
 8   eco_category         107934 non-null  object 
 9   lease_commence_date  107934 non-null  int64  
 10  latitude             107934 non-null  float64
 11  longitude            107934 non-null  float64
 12  elevation            107934 non-null  float64
 13  subzone              107934 non-null  object 
 14  planning_area        107934 non-null  object 
 15  region           

In [41]:
# remove eco_category, elevation, block, region, and planning_area
irrel_redun_columns = ['eco_category', 'elevation', 'block', 'region', 'planning_area']
df = df.drop(irrel_redun_columns, axis=1)

In [42]:
# clean the ambiguity of column 'flat_type' and 'street_name'
df['flat_type'] = df['flat_type'].replace({'1 room': '1-room', '2 room': '2-room', '3 room': '3-room', '4 room': '4-room', '5 room': '5-room'})
df['street_name'] = df['street_name'].str.title()

In [43]:
# save the test_cleaned.csv
df.to_csv('test_cleaned.csv', index = False)

Borrow some help from auxiliary data

In [55]:
# calculate distance matrix for different auxiliary data
def location_attr(df, df_target):
    df_lat, df_lng = df['latitude'].to_numpy(), df['longitude'].to_numpy()
    df_target_lat, df_target_lng = df_target['lat'].to_numpy(), df_target['lng'].to_numpy()
    distances_mat = np.zeros((len(df), len(df_target)))
    for i in range(len(df)):
        loc_house=(df_lat[i], df_lng[i])
        for j in range(len(df_target)):
            loc_target=(df_target_lat[j], df_target_lng[j])
            res = distance.distance(loc_house, loc_target).km
            distances_mat[i][j] = res
    df_distance = pd.DataFrame(distances_mat, columns=[df_target['name'].values])
    return distances_mat, df_distance

In [56]:
# count the number of target feature
def create_target_num(distances, lower, upper, name_str):
    target_num = np.zeros((distances.shape[0]))
    for r in range(distances.shape[0]):
        for c in range(distances.shape[1]):
            if lower<=distances[r][c]<=upper: target_num[r] += 1
    col_name = name_str + str(lower)+'to'+str(upper)
    df_target_num = pd.DataFrame(target_num, columns=[col_name])
    return df_target_num

In [57]:
# import all auxiliary features automatically
# 'pip install geopy' first to convert location
aux_files = ['sg-primary-schools','sg-secondary-schools','sg-train-stations','sg-gov-markets-hawker-centres','sg-shopping-malls','sg-commerical-centres']
target_names = ['primary_school_','sec_school_','train_','hawker_','shopping_','cc_']
df_cleaned = pd.read_csv('data/train_cleaned.csv', sep=',')
df_test_cleaned = pd.read_csv('data/test_cleaned.csv', sep=',')
for i in range(len(aux_files)):
    print(aux_files[i])
    df_target = pd.read_csv('data/auxiliary-data/'+aux_files[i]+'.csv', sep=',') 
    distances_mat, df_distance =location_attr(df_cleaned, df_target)
    df_distance.to_csv('dm_'+aux_files[i]+'.csv', index = False) 
    target_name = target_names[i]
    num_tar_01 = create_target_num(distances_mat, 0, 1, target_name)
    num_tar_01.to_csv(target_name+'01.csv', index = False)
    num_tar_02 = create_target_num(distances_mat, 0, 2, target_name)
    num_tar_02.to_csv(target_name+'02.csv', index = False)
    num_tar_03 = create_target_num(distances_mat, 0, 3, target_name)
    num_tar_03.to_csv(target_name+'03.csv', index = False)
    
    
    distances_mat_test, df_distance_test =location_attr(df_test_cleaned, df_target)
    df_distance_test.to_csv('dm_'+aux_files[i]+'_test.csv', index = False) 
    num_tar_01_test = create_target_num(distances_mat_test, 0, 1, target_name)
    num_tar_01_test.to_csv(target_name+'01_test.csv', index = False)
    num_tar_02_test = create_target_num(distances_mat_test, 0, 2, target_name)
    num_tar_02_test.to_csv(target_name+'02_test.csv', index = False)
    num_tar_03_test = create_target_num(distances_mat_test, 0, 3, target_name)
    num_tar_03_test.to_csv(target_name+'03_test.csv', index = False)

sg-primary-schools
sg-secondary-schools
sg-train-stations
sg-gov-markets-hawker-centres
sg-shopping-malls
sg-commerical-centres
